# Lights model with simulated dataset

1 Library
==================

In [1]:
# Library setup
%reset -f
%matplotlib inline
import os
os.environ['R_HOME'] = "/Library/Frameworks/R.framework/Versions/4.0/Resources"
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from lifelines.utils import concordance_index as c_index_score
from lights.inference import prox_QNMCEM
from lights.base.utils import visualize_vect_learning, plot_history
import numpy as np
from sklearn.preprocessing import StandardScaler
from lifelines import KaplanMeierFitter
import matplotlib.pyplot as plt
import seaborn as sns
from lifelines.utils import concordance_index as c_index_score
from prettytable import PrettyTable
from time import time
from lights.competing_methods.all_model import load_data, extract_lights_feat, extract_R_feat
from matplotlib import rc
rc('text', usetex=True)
%matplotlib inline

def printmd(string):
    display(Markdown(string))
    
import rpy2.robjects as robjects
import warnings
%load_ext rpy2.ipython

2 Simulated data
==================

In [2]:
t = PrettyTable(['Algos', 'C_index', 'time'])
test_size = .2
simu = True
data, data_lights, time_dep_feat, time_indep_feat = load_data(simu)
id_list = data_lights["id"]
nb_test_sample = int(test_size * len(id_list))
id_test = np.random.choice(id_list, size=nb_test_sample, replace=False)
data_lights_train = data_lights[data_lights.id.isin(id_test)]
data_lights_test = data_lights[data_lights.id.isin(id_test)]
X_lights_train, Y_lights_train, T_train, delta_train = \
    extract_lights_feat(data_lights_train, time_indep_feat, time_dep_feat)
X_lights_test, Y_lights_test, T_test, delta_test = \
    extract_lights_feat(data_lights_test, time_indep_feat, time_dep_feat)

data_train = data[data.id.isin(id_test)]
data_test = data[data.id.isin(id_test)]
data_R_train, T_R_train, delta_R_train = extract_R_feat(data_train)
data_R_test, T_R_test, delta_R_test = extract_R_feat(data_test)

-----------------------------------------------------------
Launching simulation using SimuJointLongitudinalSurvival...
Done simulating using SimuJointLongitudinalSurvival in 4.87e-01 seconds.


3 Competing models
==================

3.1 Penalized Cox model with time-independent features.
-------------------------------------------------------

The first model we consider as a baseline is the well known Cox PH model
with time-independent features. In this model introduced in
{Cox1972JRSS}, a parameter vector $\beta$ is estimated by minimizing the
partial log-likelihood given by 

$$
\ell_n^{\text{cox}}(\beta) = n^{-1} \sum_{i=1}^n \delta_i \big(
x_i^\top \beta - \log \sum_{i' : t_{i'} \geq t_i}
\text{exp}(x_{i'}^\top \beta) \big).
$$

We use respectively the **R** packages *survival* and *glmnet*
{simon2011regularization} for the partial log-likelihood and the
minimization of the following quantity

$$
- \ell_n^{\text{cox}}(\beta) + \xi \big( (1-\eta)||\beta||_1
+ \frac{\eta}{2} ||\beta||_2^2 \big),
$$

where $\xi$ is chosen by the a 10-fold cross-validation procedure, for a given $\eta \in [0, 1]$. Ties are handled
via the Breslow approximation of the partial likelihood . We also choose to include basic time-independent features extracted from longitudinal processes, that is values of longitudinal processes at time $t_{max}$ for each time-dependant feature.

In [3]:
# The penalized Cox model.
robjects.r.source(os.getcwd() + "/lights/competing_methods/CoxNet.R")
X_R_train = robjects.r["Cox_get_long_feat"](data_R_train, time_dep_feat,
                                          time_indep_feat)
X_R_test = robjects.r["Cox_get_long_feat"](data_R_test, time_dep_feat,
                                         time_indep_feat)
best_lambda = robjects.r["Cox_cross_val"](X_R_train, T_R_train, delta_R_train)
start = time()
trained_CoxPH = robjects.r["Cox_fit"](X_R_train, T_R_train,
                                      delta_R_train, best_lambda)
Cox_pred = robjects.r["Cox_score"](trained_CoxPH, X_R_test)
Cox_marker = np.array(Cox_pred[:])
Cox_c_index = c_index_score(T_test, Cox_marker, delta_test)
Cox_c_index = max(Cox_c_index, 1 - Cox_c_index)
Cox_exe_time = time() - start

R[write to console]: Le chargement a nécessité le package : survival

R[write to console]: Le chargement a nécessité le package : parallel

R[write to console]: Le chargement a nécessité le package : Matrix

R[write to console]: Loaded glmnet 4.1-1



Be patient, hlme is running ... 
The program took 0.06 seconds 
Be patient, hlme is running ... 
The program took 0.06 seconds 
Be patient, hlme is running ... 
The program took 0.06 seconds 
Be patient, hlme is running ... 
The program took 0.07 seconds 
Be patient, hlme is running ... 
The program took 0.06 seconds 
Be patient, hlme is running ... 
The program took 0.06 seconds 
Be patient, hlme is running ... 
The program took 0.05 seconds 
Be patient, hlme is running ... 
The program took 0.06 seconds 
Be patient, hlme is running ... 
The program took 0.08 seconds 
Be patient, hlme is running ... 
The program took 0.06 seconds 


3.2 Multivariate joint latent class model.
------------------------------------------

Among joint modeling approaches, a multivariate version of JLCMs is considered. Several longitudinal markers and time-to-event data are jointly fitted assuming that that the behavior of each response is determined by belonging to a latent homogeneous subpopulation. Contrary to the lights model, there are no shared associations between the
longitudinal models and the survival model. Given the group membership, each submodel are assumed to be independent. Therefore, the predictive marker is 

$$
\hat{\mathcal{R}}_{ik}^{(lcmm)} = \frac{\pi_{k}(\hat \theta)
\hat f(t^{max}_i \| G_i=k ; \hat \theta)\hat f(y_i | G_i=k ;
\hat \theta)}{\sum_{k=0}^{K-1} \pi_{k}(\hat \theta) \hat
f(t^{max}_i \| G_i=k ; \hat \theta)\hat f(y_i \| G_i=k ; \hat
\theta)}, 
$$

where each quantities are already defined
assuming only latent class approach. The multivariate JLCM is
implemented by {proust2017lcmm} in the **R** package *lcmm*.

In [4]:
# Multivariate joint latent class model.
start = time()
robjects.r.source(os.getcwd() + "/lights/competing_methods/MJLCMM.R")
trained_long_model, trained_mjlcmm = robjects.r["MJLCMM_fit"](data_R_train,
                                     robjects.StrVector(time_dep_feat),
                                     robjects.StrVector(time_indep_feat),
                                     alpha=2)
MJLCMM_pred = robjects.r["MJLCMM_score"](trained_long_model,
                                         trained_mjlcmm,
                                         time_indep_feat, data_R_test)
MJLCMM_marker = np.array(MJLCMM_pred.rx2('pprob')[2])
MJLCMM_c_index = c_index_score(T_test, MJLCMM_marker, delta_test)
MJLCMM_c_index = max(MJLCMM_c_index, 1 - MJLCMM_c_index)
MJLCMM_exe_time = time() - start

Be patient, multlcmm is running ... 
The program took 0.04 seconds 
The program took 534.84 seconds 
The program took 0.06 seconds 


3.3 Multivariate shared random effect model
-------------------------------------------

In shared random effect model approach, we considered multivariate joint models for multiple longitudinal outcomes and an event time. Several **R** package exist: *JMbayes*, *rstanarm* and *joineRML* for instance. For reasons of computational cost, flexibility and visibility, we use the *JMbayes* package {2017\_JMbayes}. It allows the estimation of the *lights* model without penalty and assuming that the population is homogeneous (i.e. without assuming latent classes). For a new subject $i$, the predictive marker associated with this model is the dynamic prediction of the conditional survival at time $t+s$ given the suject has survived up to time $t$ and his longitudinal measurements until time $t$. The probability is defined by: 

$$
\Pr\left(T_i^\star>t+s\vert T_i^\star>t,
\mathcal{Y}_{i}(t),\mathcal{D}_n; {\theta}\right)
$$

where 
$\mathcal{Y}_{i}(t)=\{\mathcal{Y}_{i}^1(t),\ldots,\mathcal{Y}_{i}^L(t)\}$ with $\mathcal{Y}_{i}^l(t_{ij}^l)=\{
y_{i}^{l}(t_{ij}^l);0\leq t_{ij}^l\leq t, j=1,\ldots,n_i^l\}$, $\mathcal{D}_n$ is all data from which
the model is estimated, $\theta$ the vector of parameters. The probability is approximated using Monte Carlo technique {2017\_JMbayes}.

In [5]:
# Multivariate shared random effect model.
start = time()
robjects.r.source(os.getcwd() + "/lights/competing_methods/JMBayes.R")
trained_JMBayes = robjects.r["fit"](data_R_train,
                                    robjects.StrVector(time_dep_feat),
                                    robjects.StrVector(time_indep_feat))
# JMBayes_pred = robjects.r["score"](trained_JMBayes, data_R_test, t_max=4)
# JMBayes_marker = np.array(JMBayes_pred.rx2('full.results')[0])
JMBayes_marker = np.array(robjects.r["score"](trained_JMBayes, data_R_test, t_max=4))
JMBayes_c_index = c_index_score(T_test, JMBayes_marker, delta_test)
JMBayes_c_index = max(JMBayes_c_index, 1 - JMBayes_c_index)
JMBayes_exe_time = time() - start

R[write to console]: Le chargement a nécessité le package : nlme

R[write to console]: 
Attachement du package : ‘nlme’


R[write to console]: The following objects are masked from ‘package:lcmm’:

    fixef, ranef


R[write to console]: Le chargement a nécessité le package : doParallel

R[write to console]: Le chargement a nécessité le package : foreach

R[write to console]: Le chargement a nécessité le package : iterators

R[write to console]: Le chargement a nécessité le package : rstan

R[write to console]: Le chargement a nécessité le package : StanHeaders

R[write to console]: Le chargement a nécessité le package : ggplot2

R[write to console]: rstan (Version 2.21.2, GitRev: 2e1f913d3ca3)

R[write to console]: For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)



4 Lights
==================

In [6]:
# lights
start = time()
fixed_effect_time_order = 1
learner = prox_QNMCEM(fixed_effect_time_order=fixed_effect_time_order,
                      max_iter=5, initialize=True, print_every=1,
                      compute_obj=True, simu=False,
                      asso_functions=["lp", "re"],
                      l_pen_SGL=0.02, eta_sp_gp_l1=.9, l_pen_EN=0.02)
learner.fit(X_lights_train, Y_lights_train, T_train, delta_train)
lights_marker = learner.predict_marker(X_lights_test, Y_lights_test)
lights_c_index = c_index_score(T_test, lights_marker, delta_test)
lights_c_index = max(lights_c_index, 1 - lights_c_index)
lights_exe_time = time() - start

Launching the solver prox_QNMCEM...
Launching the solver MLMM...
Launching the solver ULMM...
Done solving using ULMM in 1.86e-01 seconds


/Users/acsi/Working/lights/.env385/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/Users/acsi/Working/lights/.env385/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/Users/acsi/Working/lights/.env385/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/Users/acsi/Working/lights/.env385/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, Conv

 n_iter  |   obj    | rel_obj 
       0 |  1274.06 |      inf
       1 |  984.925 | 2.27e-01
       2 |  974.025 | 1.11e-02
       3 |   969.54 | 4.60e-03
       4 |  967.113 | 2.50e-03
       5 |  965.605 | 1.56e-03
Done solving using MLMM in 2.26e-01 seconds
 n_iter  |   obj    | rel_obj 
       0 |   201.49 |      inf
       1 |  150.459 | 2.53e-01
       2 |  107.243 | 2.87e-01
       3 |  107.352 | 1.01e-03
       4 |  105.917 | 1.34e-02
       5 |  106.837 | 8.68e-03
Done solving using prox_QNMCEM in 1.42e+01 seconds


5 Performance
==================

In [7]:
t = PrettyTable(['Algos', 'C-Index', 'time'])
t.add_row(["Cox", "%g" % Cox_c_index, "%.3f" % Cox_exe_time])
t.add_row(["MJLCMM", "%g" % MJLCMM_c_index, "%.3f" % MJLCMM_exe_time])
t.add_row(["JMBayes", "%g" % JMBayes_c_index, "%.3f" % JMBayes_exe_time])
t.add_row(["lights", "%g" % lights_c_index, "%.3f" % lights_exe_time])
print(t)

+---------+----------+---------+
|  Algos  | C-Index  |   time  |
+---------+----------+---------+
|   Cox   |    1     |  0.004  |
|  MJLCMM | 0.543478 | 534.994 |
| JMBayes | 0.573913 |  64.647 |
|  lights | 0.804348 |  15.527 |
+---------+----------+---------+
